# **v0.0**

*   Basic Starter Code






# Hugging Face login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/lib/python3.11/getpass.

# Importing Libraries

In [ ]:
from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

# Creating Agents

## Lawyer Agents

In [ ]:
# Class Implementation:
class LawyerAgent:

    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []  # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


# System prompts:
DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

# Creating the two-agents Instances:
defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

### Example Situation: State v/s Doe

In [ ]:
print("==== Opening statements ====\n")

case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)

# Prosecutor goes first
p_open = prosecution.respond(
    f"Opening statement to the Court. Background: {case_background}"
)
print("PROSECUTOR:", p_open, "\n")

# Defense responds
d_open = defense.respond(
    "Opening statement to the Court responding to the prosecution."
)
print("DEFENSE   :", d_open, "\n")

# Prosecutor rebuttal
p_rebut = prosecution.respond(
    "Brief rebuttal to the defense's opening."
)
print("PROSECUTOR:", p_rebut, "\n")


==== Opening statements ====

PROSECUTOR: "Ladies and Gentlemen of the jury, today, you will be presented with clear and convincing evidence that John Doe, my client's former employer, a company dedicated to innovation and hard work, has been the victim of a serious infringement of their rights. This crime is not just about stolen intellectual property; it's about the fundamental principles of fairness and respect for another's hard work that underpin our society.


John Doe, a skilled software engineer, played a critical role in developing proprietary algorithms essential to the success of my client's business. These algorithms are not only the result of countless hours of dedication but also a form of intellectual property protected under the law. On the day before John Doe left his job, there were large downloads from the company's servers, indicating he may have accessed confidential information.


My client, a pioneer in its field, has invested heavily in research and development.